# Work in Progress

### End-to-end a pretrained/prebuilt model from a github repo into `scivision`

Provides an example of end-to-end pipeling from the following test repo as a scivision Model:
https://github.com/acocac/scivision-treecrown-plugin
   
The code calls the repo from the catalog and run end-to-end example using Detectron pretrained/prebuilt models.

## Load libraries

In [67]:
from scivision.io import load_dataset, load_pretrained_model
from intake_zenodo_fetcher import download_zenodo_files_for_entry

## Catalog

In [68]:
cat = load_dataset('https://github.com/acocac/scivision-forest-datasets/drone_tropics.yml')

In [69]:
download_zenodo_files_for_entry(cat['sepilok_rgb'], force_download=False)

[PosixPath('Sep_2014_RGB_602500_646600.tif')]

In [70]:
tc_rgb  = cat["sepilok_rgb"]

In [74]:
img_drone = cat.sepilok_rgb().to_dask()


## Model settings and config

In [77]:
# Load model
scivision_yml = 'https://github.com/acocac/scivision-treecrown-plugin/.scivision-config.yaml'
model = load_pretrained_model(scivision_yml, allow_install=True)

INFO:scivision.koala:Call='load_pretrained_model', module='scivision.io.reader', signature=(path: os.PathLike, branch: str = 'main', allow_install: bool = False, *args, **kwargs) -> scivision.io.wrapper.PretrainedModel, kwargs_given='allow_install'
ERROR:scivision.koala:Command '['/usr/bin/python3', '-m', 'pip', 'install', 'git+https://github.com/acocac/scivision-treecrown-plugin@main#egg=src']' returned non-zero exit status 1.


CalledProcessError: ignored

In [ ]:
# let's explore the model object
model

scivision.PretrainedModel( 
  module='scivision_test_plugin', 
  model='ImageNetModel', 
  source='https://github.com/quantumjot/scivision-test-plugin.git' 
  pipe='DataPipe(input=<Parameter "X: numpy.ndarray">, output=<Parameter "image: numpy.ndarray">)' 
)

## Load input image

In [ ]:
# libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from detectron2.utils.visualizer import Visualizer, ColorMode
from PIL import Image
from IPython.display import display

In [ ]:
minx = 602500
miny = 646600

R = tc_rgb[0]
G = tc_rgb[1]
B = tc_rgb[2]

# stack up the bands in an order appropriate for saving with cv2, then rescale to the correct 0-255 range for cv2

# you will have to change the rescaling depending on the values of your tiff!
rgb = np.dstack((R, G, B))  # BGR for cv2
rgb_rescaled = 255 * rgb / 65535  # scale to image

# save this as png, named with the origin of the specific tile - change the filepath!
filepath = 'tile_' + str(minx) + '_' + str(miny) + '.png'
cv2.imwrite(filepath, rgb_rescaled)

X = cv2.imread(filepath)

y = model.predict(X)

v = Visualizer(X[:, :, ::-1], scale=1.5, instance_mode=ColorMode.IMAGE_BW)   # remove the colors of unsegmented pixels
v = v.draw_instance_predictions(y["instances"].to("cpu"))
image = cv2.cvtColor(v.get_image()[:, :, :], cv2.COLOR_BGR2RGB)
display(Image.fromarray(image))